<a href="https://colab.research.google.com/github/riccardocappi/Text-Adversarial-Attack/blob/adversarial-training/adversarial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 29.0 MB/s eta 0:00:00
  Using cached

## Imports

In [1]:
import textattack
import transformers
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.attack_recipes import DeepWordBugGao2018

2024-05-15 11:37:42.574272: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 11:37:49.124686: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Helper methods

In [2]:
class FixedHuggingFaceDataset(HuggingFaceDataset):
    def __init__(self, name_or_dataset, subset=None, split="train", dataset_columns=None, label_map=None,
                 label_names=None, output_scale_factor=None, shuffle=False, seed=69, subset_size=None, offset=0):
        super().__init__(name_or_dataset=name_or_dataset, subset=subset, split=split, dataset_columns=dataset_columns,
                         label_map=label_map, label_names=label_names, output_scale_factor=output_scale_factor,
                         shuffle=shuffle)
        if shuffle:
            self._dataset = self._dataset.shuffle(seed=seed).flatten_indices()
        if subset_size is not None:
            self._dataset = self._dataset.skip(offset).take(subset_size)

# Loading model

In [3]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Loading dataset

In [3]:
subset_train = FixedHuggingFaceDataset("imdb", split="train", subset_size=1024, shuffle=True)
subset_eval = FixedHuggingFaceDataset("imdb", split="test", subset_size=64, shuffle=True)

textattack: Loading datasets dataset imdb, split train.
textattack: Loading datasets dataset imdb, split test.


# Adversarial training DeepWordBugGao2018

In [5]:
attack = DeepWordBugGao2018.build(model_wrapper)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


### Attack before model training

In [6]:
attack_args = textattack.AttackArgs(num_examples=-1, parallel=True, disable_stdout=True)
attacker = Attacker(attack, subset_eval, attack_args)
adv_exp_bae_bert_imdb = attacker.attack_dataset()

textattack: Running 1 worker(s) on 1 GPU(s).
2024-05-13 21:49:02.182939: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 21:49:02.840362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
textattack: Worklist size: 64
textattack: Worklist candidate size: 0
  0%|          | 0/64 [00:00<?, ?it/s]2024-05-13 21:49:04.910140: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 21:49:04.911141: I external/local_xla/xla/stream_executor/cuda/cud

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 41 / 16 / 7 / 64: 100%|██████████| 64/64 [09:09<00:00,  8.58s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 41     |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 89.06% |
| Accuracy under attack:        | 25.0%  |
| Attack success rate:          | 71.93% |
| Average perturbed word %:     | 5.94%  |
| Average num. words per input: | 233.23 |
| Avg num queries:              | 234.74 |
+-------------------------------+--------+


### Define adversarial training hyperparams

In [6]:
training_args = textattack.TrainingArgs(
    num_epochs=5,
    num_clean_epochs=0,
    num_train_adv_examples=128,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=64,
    per_device_eval_batch_size=1,
    attack_epoch_interval=2,
    log_to_tb=True,
    parallel=True
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    subset_train,
    subset_eval,
    training_args
)

### Adversarial training

In [ ]:
trainer.train()

textattack: Writing logs to ./outputs/2024-05-14-23-40-57-119163/train_log.txt.
textattack: Wrote original training args to ./outputs/2024-05-14-23-40-57-119163/training_args.json.
/home/toldo/Desktop/UniPD/NLP/Text-Adversarial-Attack/venv/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 1024
textattack:   Num epochs = 5
textattack:   Num clean epochs = 0
textattack:   Instantaneous batch size per device = 1
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 64
textattack:   Gradient accumulation steps = 64
textattack:   Total optimization steps = 90
textattack: ==========================================================
textattack: Epoch 1
tex

# Check trained model accuracy

In [7]:
fine_tuned_model = transformers.AutoModelForSequenceClassification.from_pretrained("./outputs/2024-05-14-23-40-57-119163/last_model")
fine_tuned_tokenizer = transformers.AutoTokenizer.from_pretrained("./outputs/2024-05-14-23-40-57-119163/last_model")
model_wrapper_fine_tuned = textattack.models.wrappers.HuggingFaceModelWrapper(fine_tuned_model, fine_tuned_tokenizer)
fine_tuned_attack = DeepWordBugGao2018.build(model_wrapper_fine_tuned)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [8]:
fine_tuned_attack_args = textattack.AttackArgs(num_examples=-1, parallel=True, disable_stdout=True)
fine_tuned_attacker = Attacker(fine_tuned_attack, subset_eval, fine_tuned_attack_args)
fine_tuned_eval = fine_tuned_attacker.attack_dataset()

textattack: Running 1 worker(s) on 1 GPU(s).
2024-05-15 11:44:31.222721: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 11:44:31.873649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
textattack: Worklist size: 64
textattack: Worklist candidate size: 0
  0%|          | 0/64 [00:00<?, ?it/s]2024-05-15 11:44:34.119982: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-15 11:44:34.228770: I external/local_xla/xla/stream_executor/cuda/cud

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 40 / 17 / 7 / 64: 100%|██████████| 64/64 [08:44<00:00,  8.20s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 40     |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 89.06% |
| Accuracy under attack:        | 26.56% |
| Attack success rate:          | 70.18% |
| Average perturbed word %:     | 5.61%  |
| Average num. words per input: | 233.23 |
| Avg num queries:              | 220.26 |
+-------------------------------+--------+
